In [74]:
import polars as pl
from sklearn.decomposition import PCA

df = pl.read_csv("../Data/Titanic/train.csv")

In [75]:
def standardize(column):
    return (column - column.mean()) / column.std()

df = df.with_columns(
    (pl.col("Parch") + pl.col("SibSp") + 1).alias("FamilySize"),
    pl.when(pl.col("Age") < 16).then(1).otherwise(0).alias("IsChild"),
    pl.when(pl.col("Cabin").is_not_null()).then(1).otherwise(0).alias("HasCabinRegistered"),
    pl.col("Fare").log1p().alias("Fare"),
    ).with_columns(
        pl.when((pl.col("FamilySize") >= 2) & (pl.col("FamilySize") <= 4)).then(1).otherwise(0).alias("HasSmallFamily"),
        standardize(pl.col("Fare")).alias("Fare")
        ).to_dummies([
            "Sex", 
            "Embarked"
            ]).drop([
                "Sex_male",
                "Parch",
                "SibSp",
                "Name",
                "Age",
                "Ticket",
                "Cabin",
                "Embarked_S",
                "Embarked_Q",
                "Embarked_null",
                "FamilySize"
                ], 
                strict=False
                )

pca = PCA().fit_transform(df["Pclass", "HasCabinRegistered", "Fare"])

df = df.with_columns(
    pl.lit(pca[:, 0]).alias("PC1"),
    # pl.lit(pca[:, 1]).alias("PC2"),
    pl.lit(pca[:, 2]).alias("PC3")
).drop("Pclass", "HasCabinRegistered", "Fare")

In [76]:
df.write_csv("../Data/Titanic/advancedDataframeSolution.csv")